In [1]:
import pandas as pd
import numpy as np

In [87]:
df  = pd.read_csv("car_dataset.csv")
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [91]:
df.Owner.unique()

array([0, 1, 3], dtype=int64)

In [34]:
df.isnull().sum(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


(Car_Name         0
 Year             0
 Selling_Price    0
 Present_Price    0
 Kms_Driven       0
 Fuel_Type        0
 Seller_Type      0
 Transmission     0
 Owner            0
 dtype: int64,
 None)

In [35]:
df = df.drop("Car_Name", axis=1)
df["Year"] = df["Year"].max()-df["Year"]
df = pd.get_dummies(df,columns=["Fuel_Type",	"Seller_Type",	"Transmission"],drop_first=True)
df

,Year,Selling_Price,Present_Price,Kms_Driven,Owner,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Individual,Transmission_Manual
0,4,3.35,5.59,27000,0,0,1,0,1
1,5,4.75,9.54,43000,0,1,0,0,1
2,1,7.25,9.85,6900,0,0,1,0,1
3,7,2.85,4.15,5200,0,0,1,0,1
4,4,4.60,6.87,42450,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
296,2,9.50,11.60,33988,0,1,0,0,1
297,3,4.00,5.90,60000,0,0,1,0,1
298,9,3.35,11.00,87934,0,0,1,0,1
299,1,11.50,12.50,9000,0,1,0,0,1


In [36]:
X,y = df.drop("Selling_Price",axis=1), df["Selling_Price"]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [37]:
scaler = MinMaxScaler().fit(X)

In [38]:
X=scaler.transform(X)

In [39]:
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(X,y)
model.score(X,y)

0.999984513030636

In [40]:
y_pred= model.predict(X)

In [41]:
df["y_pred"] = y_pred

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

In [43]:
"r2",r2_score(y,y_pred), "rmse", mean_absolute_error(y,y_pred)**.5

('r2', 0.999984513030636, 'rmse', 0.11869270403574965)

In [44]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression(normalize=True) # 
lr_model.fit(X,y)
lr_model.score(X,y)

0.8825741581640659

In [50]:
import joblib
joblib.dump(model,open("xgb_model.joblib","wb"))
joblib.dump(scaler,open("scaler.joblib","wb"))
joblib.dump(list(df.drop(["Selling_Price","y_pred"],axis=1).columns),open("columns.joblib","wb"))

In [62]:
sample_one = [{
"Year":2014,
"Selling_Price":3.35,
"Present_Price":5.59,
"Kms_Driven":27000,
"Fuel_Type":"Petrol",
"Seller_Type":"Dealer",
"Transmission":"Manual",
"Owner":0
    }]

In [ ]:
columns = joblib.load("columns.joblib")
columns

In [63]:
df_s = pd.DataFrame(sample_one)
df_s

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0


In [57]:
df_s["Year"] = 2018-df_s["Year"]
df_s = pd.get_dummies(df_s).reindex(columns=columns, fill_value=0)
df_s

,Year,Present_Price,Kms_Driven,Owner,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Individual,Transmission_Manual
0,4,5.59,27000,0,0,1,0,1


In [58]:
scaler = joblib.load(open("scaler.joblib","rb"))
model = joblib.load(open("xgb_model.joblib","rb"))
df_s = scaler.transform(df_s)
res = model.predict(df_s)
res

array([3.3797064], dtype=float32)

In [92]:
sample_two = [{
"Year":2014,
"Present_Price":5.59,
"Kms_Driven":27000,
"Fuel_Type":"Petrol",
"Seller_Type":"Dealer",
"Transmission":"Manual",
"Owner":0
    },
{
"Year":2014,
"Present_Price":5.59,
"Kms_Driven":27000,
"Fuel_Type":"Petrol",
"Seller_Type":"Dealer",
"Transmission":"Manual",
"Owner":0
    }
]

In [93]:
scaler = joblib.load(open("scaler.joblib","rb"))
model = joblib.load(open("xgb_model.joblib","rb"))
columns = joblib.load("columns.joblib")
df_two = pd.DataFrame(sample_two)
df_two["Year"] = 2018-df_two["Year"]
df_two = pd.get_dummies(df_two).reindex(columns=columns, fill_value=0)
df_two = scaler.transform(df_two)
res = model.predict(df_two)
res


array([3.3797064, 3.3797064], dtype=float32)

In [97]:
df.iloc[y_pred.argmax()]

Car_Name         land cruiser
Year                     2010
Selling_Price            35.0
Present_Price            92.6
Kms_Driven              78000
Fuel_Type              Diesel
Seller_Type            Dealer
Transmission           Manual
Owner                       0
Name: 86, dtype: object

In [ ]:
import requests

In [101]:
url = "http://127.0.0.1:5000/api"
data ={
"Year":2014,
"Present_Price":5.59,
"Kms_Driven":27000,
"Fuel_Type":"Petrol",
"Seller_Type":"Dealer",
"Transmission":"Manual",
"Owner":0
    }
response = requests.post(url, json=data)
print(response.text)

{
  "predicted price is": "$ 3.37971"
}

